<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
#Установка нужных пакетов
!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
     |████████████████████████████████| 18.5 MB 364 kB/s 
     |████████████████████████████████| 88 kB 7.8 MB/s 
     |████████████████████████████████| 1.1 MB 54.8 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=cd53623004fff2910df1325efdc2606c135229746c9532abb03f363c09256697
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=d84351a725eb1a11923b94bdc6a9c4b54e1821db3f34ce7f092e720509f9d2eb
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built umap-learn pynndescent
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gen

In [2]:
# выгружаем датасет:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

--2022-07-18 16:25:10--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2022-07-18 16:25:10--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9cf9a26d8ad345dff2f7bd0d42.dl.dropboxusercontent.com/cd/0/get/BpWBt2a5uXSYp3woTesOqsr4rTYN5r3f1g0NI8Rj9YzhVI-56aGKmMt4qlb17kj7RnGlu6Qs45TAivSTEaDbmWImOlb-HO2ILgkoPp8ojuDKCeUS4DYGBKwPpVNEm-jM2g9mUruq9WLOMGHZxrvPj-3g87F59dMtY5WAszSiyjSyAA/file?dl=1# [following]
--2022-07-18 16:25:11--  https://uc9cf9a26d8ad345dff2f7bd0d42.dl.dropboxusercontent.com/cd/0/get/BpWBt2a5uXSYp3woTesOqsr4rTYN5r3f1g0NI8Rj9YzhVI-56aGKmMt4qlb17kj7RnGlu6Qs4

In [3]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [4]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[0]))

['Can', 'I', 'get', 'back', 'with', 'my', 'ex', 'even', 'though', 'she', 'is', 'pregnant', 'with', 'another', 'guy', "'", 's', 'baby', '?']


#Задание 1: Перевести все слова в нижний регистр (NLTK) из data и добавьте как лист токенов в листе data_tok


In [9]:
data_tok[0]

['can',
 'i',
 'get',
 'back',
 'with',
 'my',
 'ex',
 'even',
 'though',
 'she',
 'is',
 'pregnant',
 'with',
 'another',
 'guy',
 "'",
 's',
 'baby',
 '?']

In [6]:
data_tok = []
for i in range(len(data)): 
  data_tok.append(tokenizer.tokenize(data[i].lower()))

#checking
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"


###Задание 2: Подсчитайте топ10 самых популярных лем в рамках data_tok


In [54]:
dict_words = {}
lemmatizer = WordNetLemmatizer()
for token in data_tok:
  for word in token:
    word_lemmatizing = lemmatizer.lemmatize(word)
    if word in dict_words.keys(): 
      dict_words[word] += 1
    else:
      dict_words[word] = 1

import pandas as pd
df1 = pd.DataFrame.from_dict(dict_words, orient='index')
df1
df1.sort_values(0, ascending = False).head(10)

,0
?,552413
the,252068
what,214798
is,185392
a,155726
i,149740
to,141788
in,139787
how,135687
of,112001


In [13]:
token

['what',
 'are',
 'the',
 'most',
 'unintentionally',
 'hilarious',
 'movies',
 'you',
 'have',
 'ever',
 'seen',
 '?']

###Задание 3: Подсчитайте количество разных слов до и после лемматизации

In [55]:
df1.shape[0]



87819

In [51]:
import nltk
nltk.download( 'wordnet')
from nltk.stem import WordNetLemmatizer 
wnl  = WordNetLemmatizer()

dict_words_lemm = {}

for token in data_tok:
  for word in token:
    word = wnl.lemmatize(word)
    if word in dict_words_lemm.keys(): 
      dict_words_lemm[word] += 1
    else:
      dict_words_lemm[word] = 1    
      
df = pd.DataFrame.from_dict(dict_words_lemm, orient='index')
df.shape[0]



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


80303

###Задание 4: Подсчитайте количество разных слов до и после стемминга

In [50]:
from nltk.stem.snowball import SnowballStemmer
snow_stemmer = SnowballStemmer(language='english')
dict_words_stem = {}

for token in data_tok:
  for word in token:
    word1 = snow_stemmer.stem(word)
    if word1 in dict_words_stem.keys(): 
      dict_words_stem[word1] += 1
    else:
      dict_words_stem[word1] = 1   
df2 = pd.DataFrame.from_dict(dict_words_stem, orient='index')
df2.shape[0]

67203

###Задание 5: Подсчитайте количество разных слов



In [52]:
dict_words1 = {}
for token in data_tok:
  for word in token:
    if word in dict_words1.keys(): 
      dict_words1[word] += 1
    else:
      dict_words1[word] = 1

df3 = pd.DataFrame.from_dict(dict_words1, orient='index')
df3.shape[0]

87819

REGEXP

https://www.programiz.com/python-programming/regex 

https://docs.python.org/3/howto/regex.html

In [1]:
import re

pattern = 'a.*s'
test_string = 'abyss'
result = re.match(pattern, test_string)

if result:
  print("Search successful.")
else:
  print("Search unsuccessful.")

Search successful.


###Задание 6: 
https://www.hackerrank.com/challenges/matching-specific-string/problem?isFullScreen=true 

###Задание 7: 
https://www.hackerrank.com/challenges/matching-whitespace-non-whitespace-character/problem?isFullScreen=true

###Задание 8: 
https://www.hackerrank.com/challenges/matching-start-end/problem?isFullScreen=true

###Задание 9: 
https://www.hackerrank.com/challenges/matching-word-boundaries/problem?isFullScreen=true

In [2]:
import re

Test_String = 'The hackerrank team is on a mission to flatten the world by restructuring the DNA of every company on the planet. We rank programmers based on their coding skills, helping companies source great programmers and reduce the time to hire. As a result, we are revolutionizing the way companies discover and evaluate talented engineers. The hackerrank platform is the destination for the best engineers to hone their skills and companies to find top engineers.'
Regex_Pattern = r'hackerrank'

match = re.findall(Regex_Pattern, Test_String)

if result:
  print("Search successful.")
else:
  print("Search unsuccessful.")

Search successful.


In [3]:
Regex_Pattern = r"\S"

import re

print(str(bool(re.search(Regex_Pattern, '12 11 15'))).lower())

true


In [6]:
Regex_Pattern = r"^\d|\S$"
import re

print(str(bool(re.search(Regex_Pattern, '0qwer.'))).lower())

true


In [9]:
Regex_Pattern = r'\b[aeiouAEIOU]'
import re

print(str(bool(re.search(Regex_Pattern, 'Found any match?'))).lower())

true


In [8]:
#ML1_1:
#https://www.hackerrank.com/challenges/capturing-non-capturing-groups/problem?isFullScreen=true

Regex_Pattern = r'[ok]{3,}'
import re

print(str(bool(re.search(Regex_Pattern, 'okokok! cya'))).lower())

true


Bag Of Words (BOW)

In [ ]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input 
    and returns a vector of size of filtered_vocab.It puts 0 if the 
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector
def unique(sequence):
    '''This functions returns a list in which the order remains 
    same and no item repeats.Using the set() function does not 
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#create a list of stopwords.You can import stopwords from nltk too
stopwords=["to","is","a"]

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]

#Write the sentences in the corpus,in our case, just two 
string1="Welcome to Great Learning , Now start learning"
string2="Learning is a good practice"

#convert them to lower case
string1=string1.lower()
string2=string2.lower()

#split the sentences into tokens
tokens1=string1.split()
tokens2=string2.split()
print(tokens1)
print(tokens2)

#create a vocabulary list
vocab=unique(tokens1+tokens2)
print(vocab)

#filter the vocabulary list
filtered_vocab=[]
for w in vocab: 
    if w not in stopwords and w not in special_char: 
        filtered_vocab.append(w)
print(filtered_vocab)

#convert sentences into vectords
vector1=vectorize(tokens1)
print(vector1)
vector2=vectorize(tokens2)
print(vector2)

['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'learning']
['learning', 'is', 'a', 'good', 'practice']
['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'is', 'a', 'good', 'practice']
['welcome', 'great', 'learning', 'now', 'start', 'good', 'practice']
[1, 1, 2, 1, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 1]


Задание 10: Реализовать Bag of words на data_tok (можно на NLTK, можно без)